# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
!pip install gmaps
import gmaps
import os

# Import API key
from api_keys import g_key

  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076101 sha256=1079ae5567a259c98f5d38015c8aa09e8ebaa7e46332986c9bb4cb72e7da6ae8
  Stored in directory: c:\users\emily\appdata\local\pip\cache\wheels\46\48\5e\7fd28c7635ca35440a00b8a06b321febdd0b12d862ddac8659
Successfully built gmaps


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_metadata = "output_data"

weather_data = pd.read_csv(weather_metadata)

weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ugoofaaru,5.6667,73.0000,81.39,79,16,12.86,MV,1610938800
1,1,Clyde River,70.4692,-68.5914,4.06,91,95,16.40,CA,1610938800
2,2,Emmett,43.8735,-116.4993,34.00,88,99,4.41,US,1610938800
3,3,Abhā,18.2164,42.5053,51.48,81,1,3.69,SA,1610938800
4,4,Ushuaia,-54.8000,-68.3000,48.58,83,68,5.93,AR,1610938800


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [6]:
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

hum_rate = weatherdf["Humidity"].astype(float)

In [8]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacationheatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

third = second.loc[second["Wind Speed"]<10,:]
third

fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
174,174,São Filipe,14.8961,-24.4956,72.14,72,0,6.60,CV,1610938800
239,239,Morón,-34.6534,-58.6198,70.05,51,0,5.68,AR,1610938800
314,314,Sarankhola,22.3082,89.7897,71.29,68,0,2.48,BD,1610938800
329,329,Caravelas,-17.7125,-39.2481,73.96,86,0,3.38,BR,1610938800
544,544,Warmbad,-28.4500,18.7333,73.49,30,0,5.70,NaN,1610938800


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']

hotel_df.head()

,City,Country,Lat,Lng
174,São Filipe,CV,14.8961,-24.4956
239,Morón,AR,-34.6534,-58.6198
314,Sarankhola,BD,22.3082,89.7897
329,Caravelas,BR,-17.7125,-39.2481
544,Warmbad,NaN,-28.4500,18.7333


In [26]:
for index, row in hotel_df.iterrows():

    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    results = response['results']
    
    try:
        hotel_name = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        
    except (KeyError, IndexError):
        print("Missing data... skipping.")
        
    print("------------")


Retrieving Results for Index 174: São Filipe.
Missing data... skipping.
------------
Retrieving Results for Index 239: Morón.
Missing data... skipping.
------------
Retrieving Results for Index 314: Sarankhola.
Missing data... skipping.
------------
Retrieving Results for Index 329: Caravelas.
Missing data... skipping.
------------
Retrieving Results for Index 544: Warmbad.
Missing data... skipping.
------------
Retrieving Results for Index 594: Calamar.
Missing data... skipping.
------------


In [27]:
hotel_df.head()

,City,Country,Lat,Lng
174,São Filipe,CV,14.8961,-24.4956
239,Morón,AR,-34.6534,-58.6198
314,Sarankhola,BD,22.3082,89.7897
329,Caravelas,BR,-17.7125,-39.2481
544,Warmbad,NaN,-28.4500,18.7333


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel_name'

In [24]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig


NameError: name 'hotel_info' is not defined